In [138]:
%matplotlib inline

import numpy as np
import random
import pandas as pd
import requests as rq
import sys
import io
import os
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import Callback


%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Data Cleaning

In [197]:

##get data
##data is already from 2012-2020
data = pd.read_csv('data_file.csv')


##preprocessing

##filter to only patriots
data = data.loc[(data['home_team'] == 'NE') | (data['away_team'] == 'NE')]

##removes 
data = data.loc[(data['desc'].str.startswith('('))]
data = data[data['play_type'] != 'qb_kneel']
data = data[data['play_type'] != 'no_play']
data = data[data['posteam'] == 'NE']
data = data[data['play_type'] != 'kickoff']
data = data[data['play_type'] != 'extra_point']
data = data.dropna(subset=['down'])
data = data.dropna(subset=['play_type'])
data = data[(data['play_type'] == 'pass') | (data['play_type'] == 'run')]




 
##fixes playtype to be strictly pass or run 
##lables scrambles as passes
data.loc[data['pass']==1,'play_type'] = 'pass'
data.loc[data.rush==1,'play_type'] = 'run'


columnsKeep = ['play_id','game_id','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success','wp']

data = data[columnsKeep]


data.to_csv('cleaneddata.csv', index=False)


/var/folders/ys/6w09q22n4f73ds9jwmgbstjc0000gn/T/ipykernel_72523/2353747606.py:3: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,213,214,218,219,220,222,224,226,233,234,235,236,237,238,248,249,253,254,255,260,262,263,266,267,268,269,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_file.csv')


In [198]:
##pass vs run plays
print(len(data))
# Count occurrences of each play type
runCount = (data['play_type'] == 'run').sum()
passCount = (data['play_type'] == 'pass').sum()

print("Number of 'run' plays:", runCount)
print("Number of 'pass' plays:", passCount)

10896
Number of 'run' plays: 4402
Number of 'pass' plays: 6494


In [199]:
##check for nan values in data
for index, row in data.iterrows():
    if pd.isnull(row['play_type']):  # Check if the value in the 'down' column is NaN
        print(row['desc'])  # Print the value in the 'desc' column if 'down' is NaN


In [200]:
## check for missing values in data
selected_features = ['down', 'drive', 'ydstogo', 'yardline_100', 'half_seconds_remaining', 'game_seconds_remaining','wp']
missing_values_features = data[selected_features].isnull().sum()
print("Missing values in selected features:")
print(missing_values_features)

# Check for missing values in the target
missing_values_target = data['play_type'].isnull().sum()
print("\nMissing values in target (play_type):", missing_values_target)

Missing values in selected features:
down                      0
drive                     0
ydstogo                   0
yardline_100              0
half_seconds_remaining    0
game_seconds_remaining    0
wp                        0
dtype: int64

Missing values in target (play_type): 0


In [222]:
sequence_length = 500
features = data[['down','drive','ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'wp']]
target = data['play_type'].map({'pass': 0, 'run': 1})
X = []
y = []
for i in range(len(features) - sequence_length):
    X.append(features.iloc[i:i+sequence_length])
    y.append(target.iloc[i+sequence_length])  # Corrected slicing


# Convert lists to NumPy
#  arrays
X = np.array(X)
y = np.array(y)

# Split data into train and test sets
split_index = int(len(X) * 0.8)  # 80% of the data for training

# Split the data into training and testing sets
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [220]:

print(len(X_train))
nan_indices_X = np.isnan(X_train)
nan_count_X = np.sum(nan_indices_X)
print(nan_count_X)

nan_indices_X_test = np.isnan(X_test)
nan_count_X_test = np.sum(nan_indices_X_test)
print(nan_count_X_test)

nan_indices_y = np.isnan(y_train)
nan_count_y = np.sum(nan_indices_y)
print(nan_count_y)

nan_indices_Y_test = np.isnan(y_test)
nan_count_Y_test = np.sum(nan_indices_Y_test)
print(nan_count_Y_test)

print("Shape of X_train:", X_train.shape)



8708
0
0
0
0
Shape of X_train: (8708, 10, 7)


In [223]:


model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    LSTM(64, return_sequences=False),  # Return_sequences=False for the last LSTM layer
    Dense(64, activation='relu'),  # Additional dense layer
    Dropout(0.2),  # Dropout layer for regularization
    Dense(1, activation='sigmoid')  # Output layer
])


model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.005),
    metrics=['accuracy']
)



# Assuming you have already defined X_test and y_test

# Make predictions
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int).reshape(-1)

# Evaluate model performance
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_acc}')

# Print actual and predicted classes for the first 100 examples
for i in range(1000):
    actual = 'pass' if y_test[i] == 1 else 'run'  # Inverted label mapping
    predicted = 'pass' if predictions[i] == 1 else 'run'  # Inverted label mapping
    print(f'Actual: {actual}, Predicted: {predicted}')



2024-04-12 12:17:13.246160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-12 12:17:13.246804: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-12 12:17:13.247214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

65/65 [==============================] - 5s 80ms/step


2024-04-12 12:17:18.806956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-12 12:17:18.807714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-12 12:17:18.808166: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

65/65 [==============================] - 6s 78ms/step - loss: 0.6903 - accuracy: 0.5279
Test Accuracy: 0.5278846025466919
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: pass, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: pass, Predicted: pass
Actual: pass, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: run, Predicted: pass
Actual: pass, Predicted: run
Actual: run, Predicted: run
Actual: run, Predicted: run
Actual: run, Predicted: run
Actual: run, Predicted: run
Actual: run, Predicted: run
Actual: pass, Predicted: run
Actual: pass, Predicted: run
Actual: pass, Predicted: run
Actual: pass, Predicted: run
Actual: run, Predicted: run
Actual: pas